In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import warnings

warnings.filterwarnings("ignore")
gams_python_path="/Library/Frameworks/GAMS.framework/Resources/apifiles/Python/gams"

In [2]:
###import puma lookup and albuoy data

###import met2013 to PUMA2000 lookup
puma_lkup=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/msa2013_puma2000.dta')

###import albuoy data
albuoy=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/albouy_all.dta')

###limit prop by state data
# state_msa_prop_lm=state_msa_prop[state_msa_prop['prop_msa']>0]

In [49]:
puma_lkup[puma_lkup['msaname_crosswalk'].str.contains("|".join(['WY','VT','NH','MT','WV','SD','ND']))]['msaname_crosswalk'].unique()



array(['Charleston, WV', 'Boston-Cambridge-Newton, MA-NH',
       'Washington-Arlington-Alexandria, DC-VA-MD-WV',
       'Manchester-Nashua, NH', 'Bismarck, ND', 'Parkersburg-Vienna, WV',
       'Burlington-South Burlington, VT', 'Morgantown, WV'], dtype=object)

In [3]:
#fix new hampshire(33), Vermont(50), West Virginia(54), Wyoming(56), North Dakota(38), South Dakota(46), Montana(30)

####make virtual msas in albuoy data
# albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, NH']), 996,albuoy['PumaID'])
# albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, VT','Burlington, VT']), 997,albuoy['PumaID'])
# albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, WV','Charleston, WV']), 998,albuoy['PumaID'])
# albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, ND']), 1000,albuoy['PumaID'])

albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, WY','Casper, WY','Cheyenne, WY']), 999,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Rapid City, SD','Non-metro, SD','Sioux Falls, SD']), 1001,albuoy['PumaID'])
albuoy['PumaID']=np.where(albuoy['msaname'].isin(['Non-metro, MT','Great Falls, MT','Billings, MT','Missoula, MT']), 1002,albuoy['PumaID'])

####group by new virtual msas, take mean

albuoy_gr=albuoy.groupby(['PumaID','statefip'],as_index=False).mean()
####add rows to lookup
puma_lkup=puma_lkup[['met2013','msaname_crosswalk','statefip','statename','pctmsapop','PumaID']]
# puma_lkup.loc[len(puma_lkup.index)] = [996, 'NH Virtual MSA, NH', 33,'New Hampshire', 100, 996] 
# puma_lkup.loc[len(puma_lkup.index)] = [997, 'VT Virtual MSA, VT', 50,'Vermont', 100, 997] 
# puma_lkup.loc[len(puma_lkup.index)] = [998, 'WV Virtual MSA, WV', 54,'West Virginia', 100, 998] 
puma_lkup.loc[len(puma_lkup.index)] = [999, 'WY Virtual MSA, WY', 56,'Wyoming', 100, 999] 
# puma_lkup.loc[len(puma_lkup.index)] = [1000, 'ND Virtual MSA, ND', 38,'North Dakota', 100, 1000] 
puma_lkup.loc[len(puma_lkup.index)] = [1001, 'SD Virtual MSA, SD', 46,'South Dakota', 100, 1001] 
puma_lkup.loc[len(puma_lkup.index)] = [1002, 'MT Virtual MSA, MT', 30,'Montana', 100, 1002] 

###adjust lookup
puma_lkup['msaname_crosswalk_l']=puma_lkup['msaname_crosswalk'].str.lower()



In [4]:
list(puma_lkup)

['met2013',
 'msaname_crosswalk',
 'statefip',
 'statename',
 'pctmsapop',
 'PumaID',
 'msaname_crosswalk_l']

In [5]:
test=puma_lkup.groupby(['PumaID'],as_index=False).count()

In [8]:
albuoy_m=albuoy_gr.merge(puma_lkup,on=['PumaID','statefip'])


# * Get inverse distances to coast, and powers thereof
albuoy_m['mean_sea']=np.where(albuoy_m['mean_sea']<1,1,albuoy_m['mean_sea'])
albuoy_m['mean_lake']=np.where(albuoy_m['mean_lake']<1,1,albuoy_m['mean_lake'])

albuoy_m['sea']=1/albuoy_m['mean_sea']
albuoy_m['sea2']=albuoy_m['sea']**2

albuoy_m['lake']=1/albuoy_m['mean_lake']
albuoy_m['lake2']=albuoy_m['lake']**2

albuoy_m['propmsapop']=albuoy_m['pctmsapop']/100

###weight by pct_msa_pop

for col in albuoy_m.columns:
    if col not in ['statefip', 'PumaID', 'msa', 'msaname','BootstrapMSA', 'BootstrapState', 'met2013', 'msaname_crosswalk',
       'statename', 'pctmsapop', 'pctpumapop', 'dup', 'msaname_crosswalk_l',
       'propmsapop']:
        try:
            albuoy_m['%s'%col]=albuoy_m['%s'%col]*albuoy_m['propmsapop']
        except:
            pass
        
albuoy_m_gr=albuoy_m.groupby(['statefip','PumaID','msaname_crosswalk','met2013','statename'],as_index=False).sum()
# name_vars=['msaname_crosswalk_l','met2013','statefip','statename']
# num_vars=[x for x in albuoy_updated.columns if 'adj' in x ]
# albuoy_updated_lm=albuoy_updated[name_vars + num_vars]
# albuoy_gr=albuoy_updated_lm.groupby(name_vars,as_index=False).sum()
# print(len(albuoy_gr))


In [9]:
test=albuoy_m_gr.groupby('met2013',as_index=False).sum()

In [10]:
test[test['propmsapop']!=1]['propmsapop'].describe()

count    1.000000e+01
mean     1.000000e+00
std      2.340556e-16
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: propmsapop, dtype: float64

In [12]:
len(albuoy_m_gr)

1765

In [16]:
albuoy_m_gr=albuoy_m_gr.rename(columns={'msaname_crosswalk':'metarea'})
albuoy_m_gr['metarea']=albuoy_m_gr['metarea'].str.lower()
# albuoy_m_gr_lm=albuoy_m_gr[~(albuoy_m_gr['metarea'].str.contains('irtual'))]
albuoy_m_gr.to_stata('/Users/hannahkamen/Documents/population-migration2/dta/albuoy_all_weighted_v2.dta')

In [127]:
albuoy_m_gr[albuoy_m_gr['metarea'].str.contains("|".join(['wy','vt','nh','mt','wv','sd','nd']))]['metarea'].unique()

array(['gadsden, al', 'phoenix-mesa-scottsdale, az',
       'san francisco-oakland-hayward, ca',
       'san luis obispo-paso robles-arroyo grande, ca',
       'oxnard-thousand oaks-ventura, ca', 'grand junction, co',
       'norwich-new london, ct',
       'washington-arlington-alexandria, dc-va-md-wv',
       'deltona-daytona beach-ormond beach, fl',
       'orlando-kissimmee-sanford, fl', 'lakeland-winter haven, fl',
       'naples-immokalee-marco island, fl',
       'atlanta-sandy springs-roswell, ga',
       'augusta-richmond county, ga-sc',
       'indianapolis-carmel-anderson, in', 'portland-south portland, me',
       'boston-cambridge-newton, ma-nh', 'grand rapids-wyoming, mi',
       'mt virtual msa, mt', 'las vegas-henderson-paradise, nv',
       'manchester-nashua, nh', 'binghamton, ny', 'bismarck, nd',
       'cleveland-elyria, oh', 'bend-redmond, or',
       'portland-vancouver-hillsboro, or-wa',
       'greenville-anderson-mauldin, sc',
       'hilton head island-bluffto

In [68]:
albuoy_m_gr[albuoy_m_gr['metarea'].str.contains('irtual')]

,statefip,PumaID,metarea,met2013,statename,msa,Wage_orig,Wage,Price,QOL_orig,...,DeltaA1F130YearRelHum11,DeltaA1F130YearDewPt11,DeltaA1F130YearTempAvg11,DeltaA1F130YearRelHum12,DeltaA1F130YearDewPt12,DeltaA1F130YearTempAvg12,BootstrapMSA,BootstrapState,pctpumapop,proppumapop
959,30,1002.0,"mt virtual msa, mt",1002,Montana,178436.571429,-0.236539,-0.244741,-0.233551,0.041783,...,-0.174235,9.242236,9.465663,0.499331,8.926607,8.912899,207.285721,25.0,100.0,1.0
983,33,996.0,"nh virtual msa, nh",996,New Hampshire,339999.000000,-0.095919,-0.074009,-0.075374,0.023472,...,1.875408,9.141467,8.749085,0.890460,9.554008,9.453859,295.000000,28.0,100.0,1.0
1243,38,1000.0,"nd virtual msa, nd",1000,North Dakota,389999.000000,-0.251035,-0.251855,-0.585009,-0.066803,...,-1.011920,10.056675,10.653851,0.400843,10.815710,10.873327,299.000000,33.0,100.0,1.0
1463,46,1001.0,"sd virtual msa, sd",1001,South Dakota,337773.571429,-0.242468,-0.247073,-0.426008,-0.018696,...,-1.908507,9.295931,10.365475,-0.719148,8.980893,9.433858,278.714294,40.0,100.0,1.0
1656,50,997.0,"vt virtual msa, vt",997,Vermont,382825.500000,-0.149638,-0.157590,-0.096744,0.043199,...,1.595973,9.464647,9.147495,-0.020952,10.017255,10.138587,240.500000,44.0,100.0,1.0
1738,54,998.0,"wv virtual msa, wv",998,West Virginia,481434.125000,-0.212419,-0.206514,-0.496634,-0.058214,...,-1.470017,7.171453,7.923409,0.773879,6.104775,5.993682,278.125000,47.0,100.0,1.0
1762,56,999.0,"wy virtual msa, wy",999,Wyoming,285732.000000,-0.171745,-0.178350,-0.238716,0.007607,...,-3.199511,8.657593,9.881630,-0.485891,8.419120,8.696012,177.500000,49.0,100.0,1.0


In [96]:
second_stage1=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/results/temp/2nd_stage_avg_age1.dta')
second_stage1

,var,coef,stderr,tstat,pval,N,r2,string
0,hot,0.010562,0.006066,1.741359,0.082994,246,0.281795,
1,cold,-0.016338,0.008041,-2.031911,0.043343,246,0.281795,
2,lnprec,0.050117,0.322056,0.155616,0.876476,246,0.281795,
3,lnrelhum,1.138860,1.102377,1.033095,0.302674,246,0.281795,
4,lnavgtemp,-3.649001,1.985529,-1.837798,0.067417,246,0.281795,
5,lnsun,3.413161,1.486046,2.296807,0.022554,246,0.281795,
6,lnsea,-0.717088,0.454070,-1.579244,0.115691,246,0.281795,
7,lnsea2,0.357206,0.215931,1.654257,0.099476,246,0.281795,
8,lnlake,0.273951,0.346447,0.790745,0.429929,246,0.281795,
9,lnlake2,-0.029036,0.160834,-0.180532,0.856898,246,0.281795,


In [87]:
old_ccdb=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/ccdb_2007_modified.dta')

old_lookup=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/dta/0610_msa_lookup.dta')

In [91]:
old_lookup['metarea_07_lookup']=old_lookup['metarea_07_lookup'].str.replace("/","-")

In [93]:
old_lookup.to_stata('/Users/hannahkamen/Documents/population-migration2/dta/0610_msa_lookup.dta')

In [94]:
len(old_ccdb['metarea'].unique())

309

In [95]:
second_stage2=pd.read_stata('/Users/hannahkamen/Documents/population-migration2/results/temp/2nd_stage_avg_age2.dta')
second_stage2

,var,coef,stderr,tstat,pval,N,r2,string
0,hot,0.011594,0.006161,1.881790,0.061161,246,0.293206,
1,cold,-0.013541,0.008167,-1.657926,0.098732,246,0.293206,
2,lnprec,-0.074167,0.327136,-0.226715,0.820852,246,0.293206,
3,lnrelhum,1.226606,1.119767,1.095412,0.274513,246,0.293206,
4,lnavgtemp,-3.590108,2.016851,-1.780056,0.076422,246,0.293206,
5,lnsun,3.386173,1.509489,2.243258,0.025859,246,0.293206,
6,lnsea,-0.605058,0.461233,-1.311825,0.190922,246,0.293206,
7,lnsea2,0.330624,0.219338,1.507372,0.133124,246,0.293206,
8,lnlake,0.256478,0.351912,0.728815,0.466876,246,0.293206,
9,lnlake2,-0.012720,0.163371,-0.077861,0.938008,246,0.293206,


In [72]:
second_stage1

NameError: name 'second_stage1' is not defined